In [ ]:
#read-PDF imports here
!pip install PyPDF2
import PyPDF2

#pre-processing imports here
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string

In [4]:
#mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#file locations on drive
grimm_url = '/content/drive/My Drive/Lessons/storyTeller/FairytalesByTheBrothersGrimm.txt'
coraline_url = '/content/drive/My Drive/Lessons/storyTeller/Coraline.pdf'
alice_url = '/content/drive/My Drive/Lessons/storyTeller/AlicesAdvanturesInWonderland.txt'

In [ ]:
#load punctuation symbols
punct = string.punctuation

# **Pre-processing Coraline**

In [ ]:
#a function to pre process Coraline by Neil Gaiman

def preprocess_coraline(book):
  '''
  param book: url od a PDF book file
  '''
  output = ""
  data = open(book, 'rb')
  data = PyPDF2.PdfFileReader(book)
  npages = data.getNumPages()
  for i in range(npages):
    page_i = data.getPage(i).extractText()
    output += page_i
  output = output[1227:]
  output = output.lower()
  for word in output:
    for char in word:
        if char in punct:
            word = word.replace(char, "")
  remove_punct = "".join([word for word in output if word not in punct])
  processed = word_tokenize(remove_punct)
  print('Coraline database includes {} tokens, and {} unique tokens after editing'.format(len(processed), len(set(processed)))) 
  return processed

coraline = preprocess_coraline(coraline_url) 

## **Preprocessing Alice in Wonderland**

In [ ]:
#a function to pre process Alice's Advantures in Wonderland by Lewis Carroll

def load_alice(text_file, punct, not_a_word):
    '''
    param text_file: url to Project Gutenberg's text file for Alice's Advantures in Wonderland by Lewis Carroll
    param punct: a string of punctuation characters we'd like to filter
    param not_a_word: a list of words we'd like to filter
    '''
    book = open(text_file, 'r')
    book = book.read()
    book = book[715:145060]
    book_edit = re.sub('[+]', '', book)
    book_edit = re.sub(r'(CHAPTER \w+.\s)', '', book)
    words = word_tokenize(book_edit.lower())
    
    word_list = []
    
    # filtering punctuation and non-words
    for word in words:
        for char in word:
            if char in punct:
                word = word.replace(char, "")
        if word not in punct and word not in not_a_word:
            word_list.append(word)

    print('Alice database includes {} tokens, and {} unique tokens after editing'.format(len(word_list), len(set(word_list)))) 
    return word_list

alice = load_alice(alice_url, (punct.replace('-', "") + '’' + '‘'), ['s', '--', 'nt', 've', 'll', 'd'])

# **Preprocessing Grimm**


In [ ]:
def load_fairytales(text_file):
    '''
    param text_file: url to Project Gutenberg's text file for Fairytales by The Brothers Grimm
    '''
    book = open(text_file, encoding='cp1252')
    book = book.read()
    book = book[2376:519859]
    book_edit = re.sub('[(+*)]', '', book)
    words = word_tokenize(book_edit.lower())

    # filtering punctuation inside tokens (example: didn't or wow!)
    for word in words:
        for char in word:
            if char in punct:
                word = word.replace(char, "")

    # filtering punctuation as alone standing tokens(example: \ or ,)
    words = [word for word in words if word not in punct]

    print('Fairytales database includes {} tokens, and {} unique tokens after editing'.format(len(words), len(set(words))))            
    return words

brothers_grimm = load_fairytales(grimm_url)

# **Combined database including all books**

In [ ]:
data = coraline + alice + brothers_grimm
data[:10]

# **Convert Data into Numeric Values**

# **Batching Data**

# **Defining the Neural Network**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

# **Defining Training Function**

# **Train Model**

# **Save Checkpoint**

In [ ]:
checkpoint_url = '/content/drive/My Drive/Lessons/storyTeller/checkpoint3.pth'

checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'word_to_index': word_to_index,
              'index_to_word': {i: word for i, word in enumerate(vocab)},
              'epochs': epochs,
              'average_loss': average_loss,
              'device': device,
              'optimizer_state': optimizer.state_dict(),
              'batch_size': batch_size}

torch.save(checkpoint, checkpoint_url)

# **Load Checkpoint**

In [5]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.optimizer_state = checkpoint['optimizer_state']
    model.load_state_dict(checkpoint['state_dict'])
    model.device = checkpoint['device']
    model.word_to_index = checkpoint['word_to_idxx']
    model.index_to_word = checkpoint['idx_to_word']
    model.average_loss = checkpoint['average_loss']
    return model

checkpoint_url = '/content/drive/My Drive/Lessons/storyTeller/checkpoint5.pth'
model = load_checkpoint(checkpoint_url)
index_to_word = model.index_to_word
model

AttributeError: ignored

In [ ]:
import pandas as pd

loss_plot = pd.DataFrame(model.average_loss)
loss_plot.plot()

# **Predict Function**

In [ ]:
def predict(model, first_words ,story_len ,top_k):
    '''
    param model: trained model
    param first_words: a string of 5 (n_feature) words to begin the story
    param story_len: an integer symbolizing the number of words you'd like the story to have
    param top_k: the number of top probabilities per word that the network will randomly select from
    '''
    feature = (first_words.lower()).split(" ")
    for i in feature:
        story.append(i)
    for i in range(story_len):
        feature_idx = torch.tensor([word_to_index[word] for word in feature], dtype=torch.long)
        feature_idx = feature_idx.to(device)
        with torch.no_grad():
            output = model.double().forward(feature_idx)
        ps = torch.exp(output)
        topk_combined = ps.topk(top_k, sorted=True)
        #top kk probabilities
        topk_ps = topk_combined[0][0]
        #top kk classes
        topk_class = topk_combined[1][0]
        topk_class = [index_to_word[int(i)] for i in topk_class]
        next_word = random.choice(topk_class)
        feature = feature[1:]
        feature.append(next_word)
        story.append(next_word)
    return story

# **Predict**

In [ ]:
import random
first_words = input('Type the first {} words to start the story:\nexample: A lovely day at the\n'.format(batch_size))

top_k = 3
story_len = 50
story = []
device = 'cuda:0'

#Predicting and Handling User-Input Errors
try:      
    prediction = predict(model, first_words, story_len, top_k)
except KeyError as error:
    print('Oops, looks like you\'ve selected a word that the network does not understand yet: ', error)
    if story[0] != "":
        story = story[len(first_words):]
    first_words = input('please select a different word:\nexample: A lovely day at the\n')
    prediction = predict(model, first_words, story_len, top_k)
except KeyError and RuntimeError:
    if story[0] != "":
        story = story[len(first_words):]
    first_words = input('Oops, looks like you\'ve typed {} words instead of {}!\n\nType the first 5 words to start the story:\nexample: A lovely day at the\n'.format(len(first_words.split(" ")), n_features))
    prediction = predict(model, first_words, story_len, top_k)

print('-----------------------------------------------------\n The STORY \n-----------------------------------------------------')
print(" ".join(story))